# Gun Violence in the U.S. from 2013-2018

##### Team: Andres Baez, Edward Chen

## Project Overview:
***
- Reseach question, hypothesis
  - What do we hope to achieve?
- Datasets and Background & Prior Work
  - Brief summary of our datasets
  - Previous work that have been done using these datasets
- Data Cleaning and Analysis
  - Gun Violence Data 
- Discussion and Moving Forward
- Conclusion and Implications
  - What further questions we may have
  - How to improve our study


## Research Question
***
Short Answer:

Long Answer:

## Hypothesis
***

## Datasets:
***

Dataset Name:

Link to dataset:

Description: 


Dataset Name:

Link to dataset:

Description: 


Dataset Name:

Link to dataset:

Description: 

## Background and Prior Work
***

## Methods
***
Data Cleaning:

Data Analysis:

What to report:

#### Data Cleaning and Analysis per Dataset
***

## Ethics and Privacy

## Dicussion

## Moving Forward